In [1]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [3]:
import torch
X = torch.tensor(x).float()
Y = torch.tensor(y).float()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
ds = MyDataset(X, Y)
dl = DataLoader(ds, batch_size=2, shuffle=True)
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x
mynet = MyNeuralNet().to(device)

/tmp/ipython-input-3900619449.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).float()
/tmp/ipython-input-3900619449.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).float()


In [4]:
# define custom loss function
def my_mean_squared_error(pred, y):
  loss = (pred-y)**2
  loss = loss.mean()
  return loss

In [5]:
loss_fn = nn.MSELoss()
loss_value = loss_fn(mynet(X), Y)
print(loss_value)

tensor(99.1912, grad_fn=<MseLossBackward0>)


In [6]:
my_mean_squared_error(mynet(X), Y)

tensor(99.1912, grad_fn=<MeanBackward0>)

#### fetching values of intermediate layers

In [8]:
# option 1: directly calling layers as if they are functions
input_to_hidden = mynet.input_to_hidden_layer(X)
hidden_activation = mynet.hidden_layer_activation(input_to_hidden)
print(hidden_activation)

tensor([[1.5100, 0.0000, 0.5822, 0.0000, 0.0000, 0.0000, 0.8496, 0.0000],
        [3.0171, 0.0000, 0.1254, 0.0000, 0.0000, 0.0000, 2.0231, 0.0000],
        [4.5241, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 3.1967, 0.0000],
        [6.0311, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 4.3702, 0.0000]],
       grad_fn=<ReluBackward0>)


In [9]:
#option 2: specifying the layers that we want to look at in the forward method
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    def forward(self, x):
        hidden1 = self.input_to_hidden_layer(x)
        hidden2 = self.hidden_layer_activation(hidden1)
        output = self.hidden_to_output_layer(hidden2)
        return output, hidden2

In [13]:
mynet = NeuralNet().to(device)
mynet(X)[1]

tensor([[0.3101, 0.0000, 1.2160, 0.0000, 0.6691, 0.2001, 1.8434, 0.0000],
        [0.0000, 0.1055, 2.4739, 0.0000, 1.6422, 0.1629, 3.3412, 0.6846],
        [0.0000, 0.5004, 3.7318, 0.1315, 2.6153, 0.1257, 4.8390, 1.4295],
        [0.0000, 0.8954, 4.9898, 0.3722, 3.5883, 0.0885, 6.3368, 2.1745]],
       grad_fn=<ReluBackward0>)